<a href="https://colab.research.google.com/github/YIF27/Dataset-of-Spinel/blob/main/bond_analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pymatgen.core import Structure
class BondAnalyzer:
def __init__(self, input_data):
        if isinstance(input_data, Structure):
            self.structure = input_data
        elif isinstance(input_data, str):  # Assuming a file path is given as a string
            self.structure = Structure.from_file(input_data)
        else:
            raise ValueError("Input must be a pymatgen Structure or a path to a POSCAR file.")

        def get_bonds_for_atom(self, atom_index):
        """
        Get the bonds involving the atom with the specified index.

        Args:
            atom_index (int): Index of the atom in the pymatgen Structure.

        Returns:
            List[Tuple[int, str]]: List of bonds for the atom, represented as tuples containing the bonded atom's index and element type.
        """
        cnn = CrystalNN()
        neighbors = cnn.get_nn_info(self.structure, atom_index)
        bonds = [(neighbor['site_index'], neighbor['site'].species_string) for neighbor in neighbors]

        return bonds


        @staticmethod
        def generate_bond_info(self):
          composition_values = list(self.structure.composition.as_dict().values())
          metal1_number = int(composition_values[0])
          metal2_number = int(composition_values[1])
          oxygen_number = int(composition_values[2])
          metal1_coords = self.structure.frac_coords[0: metal1_number]
          metal2_coords = self.structure.frac_coords[metal1_number: metal2_number + metal1_number]
          oxygen_coords = self.structure.frac_coords[metal2_number + metal1_number:]

          R_metal1_oxygen = np.zeros((oxygen_number, metal1_number))
          PBD_O_metal1_type = np.zeros((oxygen_number, metal1_number))
          c_O_metal1 = np.zeros((oxygen_number * metal1_number, 3))
          t = 0
          for i in range(oxygen_number):
              for j in range(metal1_number):
                  r, d = compute_distances_and_displacements(oxygen_coords[i], metal1_coords[j])
                  minimum_r = min(r)
                  arg_minimum_r = int(np.argmin(r))
                  R_metal1_oxygen[i, j] = minimum_r
                  PBD_O_metal1_type[i, j] = arg_minimum_r
                  c_O_metal1[t] = d[arg_minimum_r]
                  t = t + 1

          R_metal2_oxygen = np.zeros((oxygen_number, metal2_number))
          PBD_O_metal2_type = np.zeros((oxygen_number, metal2_number))
          c_O_metal2 = np.zeros((oxygen_number * metal2_number, 3))
          t = 0
          for i in range(oxygen_number):
              for j in range(metal2_number):
                  r, d = compute_distances_and_displacements(oxygen_coords[i], metal2_coords[j])
                  minimum_r = min(r)
                  arg_minimum_r = int(np.argmin(r))
                  R_metal2_oxygen[i, j] = minimum_r
                  PBD_O_metal2_type[i, j] = arg_minimum_r
                  c_O_metal2[t] = d[arg_minimum_r]
                  t = t + 1

          metal1_oxygen_metal2_1 = []
          for i in range(oxygen_number):
              for j in range(metal1_number):
                  for k in range(metal2_number):
                      if R_metal1_oxygen[i][j] < 0.255 and R_metal2_oxygen[i][k] < 0.255:
                          metal1_oxygen_metal2_1_tmp = [i, j, k]
                          metal1_oxygen_metal2_1_tmp.append(R_metal1_oxygen[i][j])
                          d1 = R_metal1_oxygen[i][j]
                          metal1_oxygen_metal2_1_tmp.append(R_metal2_oxygen[i][k])
                          d2 = R_metal2_oxygen[i][k]
                          c1 = c_O_metal1[i * metal1_number + j][:]
                          c2 = c_O_metal2[i * metal2_number + k][:]
                          p = np.dot(c1, c2) / (d1 * d2)
                          metal1_oxygen_metal2_1_tmp.append(np.degrees(np.arccos(p)))
                          metal1_oxygen_metal2_1.append(metal1_oxygen_metal2_1_tmp)

          metal1_oxygen_metal2_2 = []
          for i in range(oxygen_number):
              for j in range(metal1_number):
                  for k in range(metal2_number):
                      if R_metal1_oxygen[i][j] > 0.402:
                          if R_metal1_oxygen[i][j] < 0.46:
                              if R_metal2_oxygen[i][k] < 0.255:
                                  metal1_oxygen_metal2_2_tmp = [i, j, k]
                                  metal1_oxygen_metal2_2_tmp.append(R_metal1_oxygen[i][j])
                                  d1 = R_metal1_oxygen[i][j]
                                  metal1_oxygen_metal2_2_tmp.append(R_metal2_oxygen[i][k])
                                  d2 = R_metal2_oxygen[i][k]
                                  c1 = c_O_metal1[i * metal1_number + j][:]
                                  c2 = c_O_metal2[i * metal2_number + k][:]
                                  p = np.dot(c1, c2) / (d1 * d2)
                                  if p < 0.2588:
                                    metal1_oxygen_metal2_2_tmp.append(np.degrees(np.arccos(p)))
                                    metal1_oxygen_metal2_2.append(metal1_oxygen_metal2_2_tmp)

          metal1_oxygen_metal2_3 = []
          for i in range(oxygen_number):
              for j in range(metal1_number):
                  for k in range(metal2_number):
                      if R_metal1_oxygen[i][j] < 0.255:
                          if R_metal2_oxygen[i][k] > 0.40:
                              if R_metal2_oxygen[i][k] < 0.46:
                                  metal1_oxygen_metal2_3_tmp = []
                                  metal1_oxygen_metal2_3_tmp.append(i)
                                  metal1_oxygen_metal2_3_tmp.append(j)
                                  metal1_oxygen_metal2_3_tmp.append(k)
                                  metal1_oxygen_metal2_3_tmp.append(R_metal1_oxygen[i][j])
                                  d1 = R_metal1_oxygen[i][j]
                                  metal1_oxygen_metal2_3_tmp.append(R_metal2_oxygen[i][k])
                                  d2 = R_metal2_oxygen[i][k]
                                  c1 = c_O_metal1[i * metal1_number + j][:]
                                  c2 = c_O_metal2[i * metal2_number + k][:]
                                  p = np.dot(c1, c2) / (d1 * d2)
                                  if p < 0.2588:
                                    metal1_oxygen_metal2_3_tmp.append(np.degrees(np.arccos(p)))
                                    metal1_oxygen_metal2_3.append(metal1_oxygen_metal2_3_tmp)

          metal2_oxygen_metal2_1 = []
          for i in range(oxygen_number):
              for j in range(metal2_number - 1):
                  for k in range(j + 1, metal2_number):
                      if R_metal2_oxygen[i][j] < 0.255:
                          if R_metal2_oxygen[i][k] < 0.255:
                              metal2_oxygen_metal2_1_tmp = []
                              metal2_oxygen_metal2_1_tmp.append(i)
                              metal2_oxygen_metal2_1_tmp.append(j)
                              metal2_oxygen_metal2_1_tmp.append(k)
                              metal2_oxygen_metal2_1_tmp.append(R_metal2_oxygen[i][j])
                              d1 = R_metal2_oxygen[i][j]
                              metal2_oxygen_metal2_1_tmp.append(R_metal2_oxygen[i][k])
                              d2 = R_metal2_oxygen[i][k]
                              c1 = c_O_metal2[i * metal2_number + j][:]
                              c2 = c_O_metal2[i * metal2_number + k][:]
                              p = np.dot(c1, c2) / (d1 * d2)
                              metal2_oxygen_metal2_1_tmp.append(np.degrees(np.arccos(p)))
                              metal2_oxygen_metal2_1.append(metal2_oxygen_metal2_1_tmp)

          metal2_oxygen_metal2_2 = []
          R_metal2_oxygen_1 = []
          R_metal2_oxygen_2 = []
          for i in range(oxygen_number):
              for j in range(metal2_number - 1):
                  for k in range(j + 1, metal2_number):
                      if (R_metal2_oxygen[i][j] < 0.255 and 0.40 < R_metal2_oxygen[i][k] < 0.46) or (
                              R_metal2_oxygen[i][k] < 0.255 and 0.40 < R_metal2_oxygen[i][j] < 0.46):
                          metal2_oxygen_metal2_2_tmp = [i, j, k]
                          metal2_oxygen_metal2_2_tmp.append(R_metal2_oxygen[i][j])
                          d1 = R_metal2_oxygen[i][j]
                          metal2_oxygen_metal2_2_tmp.append(R_metal2_oxygen[i][k])
                          d2 = R_metal2_oxygen[i][k]
                          c1 = c_O_metal2[i * metal2_number + j][:]
                          c2 = c_O_metal2[i * metal2_number + k][:]
                          p = np.dot(c1, c2) / (d1 * d2)
                          if p < 0.2588:
                              metal2_oxygen_metal2_2_tmp.append(np.degrees(np.arccos(p)))
                              metal2_oxygen_metal2_2.append(metal2_oxygen_metal2_2_tmp)
                              if d1 > d2:
                                  R_metal2_oxygen_1.append(d2)
                                  R_metal2_oxygen_2.append(d1)
                              if d2 > d1:
                                  R_metal2_oxygen_1.append(d1)
                                  R_metal2_oxygen_2.append(d2)

          metal1_oxygen_metal1_1 = []
          R_metal1_oxygen_1 = []
          R_metal1_oxygen_2 = []
          for i in range(oxygen_number):
              for j in range(metal1_number - 1):
                  for k in range(j + 1, metal1_number):
                      if (R_metal1_oxygen[i][j] < 0.255 and 0.40 < R_metal1_oxygen[i][k] < 0.46) or (
                              R_metal1_oxygen[i][k] < 0.255 and 0.40 < R_metal1_oxygen[i][j] < 0.46):
                          metal1_oxygen_metal1_1_tmp = [i,j,k]
                          metal1_oxygen_metal1_1_tmp.append(R_metal1_oxygen[i][j])
                          d1 = R_metal1_oxygen[i][j]
                          metal1_oxygen_metal1_1_tmp.append(R_metal1_oxygen[i][k])
                          d2 = R_metal1_oxygen[i][k]
                          c1 = c_O_metal1[i * metal1_number + j][:]
                          c2 = c_O_metal1[i * metal1_number + k][:]
                          p = np.dot(c1, c2) / (d1 * d2)
                          if p < 0.2588:
                              metal1_oxygen_metal1_1_tmp.append(np.degrees(np.arccos(p)))
                              metal1_oxygen_metal1_1.append(metal1_oxygen_metal1_1_tmp)
                              if d1 > d2:
                                  R_metal1_oxygen_1.append(d2)
                                  R_metal1_oxygen_2.append(d1)
                              if d2 > d1:
                                  R_metal1_oxygen_1.append(d1)
                                  R_metal1_oxygen_2.append(d2)

          metal1_oxygen_metal1_2 = []
          for i in range(oxygen_number):
              for j in range(metal1_number - 1):
                  for k in range(j + 1, metal1_number):
                      if R_metal1_oxygen[i][j] < 0.255:
                          if R_metal1_oxygen[i][k] < 0.255:
                              d1 = R_metal1_oxygen[i][j]
                              d2 = R_metal1_oxygen[i][k]
                              c1 = c_O_metal1[i * metal1_number + j][:]
                              c2 = c_O_metal1[i * metal1_number + k][:]
                              p = np.dot(c1, c2) / (d1 * d2)
                              angle = math.acos(p) * 180 / math.pi
                              metal1_oxygen_metal1_2_tmp = [i, j, k, d1, d2, angle]
                              metal1_oxygen_metal1_2.append(metal1_oxygen_metal1_2_tmp)

          metal1_oxygen_metal2_1 = np.array(metal1_oxygen_metal2_1)
          metal1_oxygen_metal2_1_new = metal1_oxygen_metal2_1[metal1_oxygen_metal2_1[:, 5].argsort()]

          metal1_oxygen_metal2_2 = np.array(metal1_oxygen_metal2_2)
          metal1_oxygen_metal2_2_new = metal1_oxygen_metal2_2[metal1_oxygen_metal2_2[:, 5].argsort()]

          metal1_oxygen_metal2_3 = np.array(metal1_oxygen_metal2_3)
          metal1_oxygen_metal2_3_new = metal1_oxygen_metal2_3[metal1_oxygen_metal2_3[:, 5].argsort()]

          metal1_oxygen_metal1_1 = np.array(metal1_oxygen_metal1_1)
          metal1_oxygen_metal1_1_new = metal1_oxygen_metal1_1[metal1_oxygen_metal1_1[:, 5].argsort()]

          metal1_oxygen_metal1_2 = np.array(metal1_oxygen_metal1_2)
          metal1_oxygen_metal1_2_new = metal1_oxygen_metal1_2[metal1_oxygen_metal1_2[:, 5].argsort()]

          metal2_oxygen_metal2_1 = np.array(metal2_oxygen_metal2_1)
          metal2_oxygen_metal2_1_new = metal2_oxygen_metal2_1[metal2_oxygen_metal2_1[:, 5].argsort()]

          metal2_oxygen_metal2_2 = np.array(metal2_oxygen_metal2_2)
          metal2_oxygen_metal2_2_new = metal2_oxygen_metal2_2[metal2_oxygen_metal2_2[:, 5].argsort()]

          metal2_oxygen_metal2_79 = count_bond_number(metal2_oxygen_metal2_2_new, 76, 80)
          metal2_oxygen_metal2_93 = count_bond_number(metal2_oxygen_metal2_1_new, 0, 94)
          metal2_oxygen_metal2_123 = count_bond_number(metal2_oxygen_metal2_1_new, 120, 126)
          metal2_oxygen_metal2_126 = count_bond_number(metal2_oxygen_metal2_2_new, 124, 127)
          metal2_oxygen_metal2_158 = count_bond_number(metal2_oxygen_metal2_2_new, 157, 160)
          metal2_oxygen_metal2_179 = count_bond_number(metal2_oxygen_metal2_2_new, 177, 180)
          metal1_oxygen_metal2_79 = count_bond_number(metal1_oxygen_metal2_3_new, 0, 80)
          metal1_oxygen_metal2_93 = count_bond_number(metal1_oxygen_metal2_1_new, 92, 94)
          metal1_oxygen_metal2_123 = count_bond_number(metal1_oxygen_metal2_1_new, 120, 126)
          metal1_oxygen_metal2_126 = count_bond_number(metal1_oxygen_metal2_3_new, 124, 127)
          metal1_oxygen_metal2_158 = count_bond_number(metal1_oxygen_metal2_3_new, 157, 160)
          metal1_oxygen_metal2_179 = count_bond_number(metal1_oxygen_metal2_3_new, 177, 180)
          metal2_oxygen_metal1_79 = count_bond_number(metal1_oxygen_metal2_2_new, 77, 81)
          metal2_oxygen_metal1_126 = count_bond_number(metal1_oxygen_metal2_2_new, 124, 127)
          metal2_oxygen_metal1_158 = count_bond_number(metal1_oxygen_metal2_2_new, 157, 160)
          metal2_oxygen_metal1_179 = count_bond_number(metal1_oxygen_metal2_2_new, 177, 180)
          metal1_oxygen_metal1_79 = count_bond_number(metal1_oxygen_metal1_1_new, 77, 81)
          metal1_oxygen_metal1_93 = count_bond_number(metal1_oxygen_metal1_2_new, 90, 95)
          metal1_oxygen_metal1_123 = count_bond_number(metal1_oxygen_metal1_2_new, 120, 125)
          metal1_oxygen_metal1_126 = count_bond_number(metal1_oxygen_metal1_1_new, 124, 127)
          metal1_oxygen_metal1_158 = count_bond_number(metal1_oxygen_metal1_1_new, 157, 160)
          metal1_oxygen_metal1_179 = count_bond_number(metal1_oxygen_metal1_1_new, 177, 180)
          bond = [metal1_oxygen_metal2_123, metal2_oxygen_metal2_123, metal1_oxygen_metal1_123, metal2_oxygen_metal1_158,
                  metal2_oxygen_metal2_158, metal1_oxygen_metal2_158, metal1_oxygen_metal1_158, metal1_oxygen_metal2_179,
                  metal2_oxygen_metal2_179, metal2_oxygen_metal1_179, metal1_oxygen_metal1_179,
                  metal1_oxygen_metal2_93, metal2_oxygen_metal2_93, metal1_oxygen_metal1_93, metal2_oxygen_metal1_126,
                  metal1_oxygen_metal2_126, metal2_oxygen_metal2_126, metal1_oxygen_metal1_126, metal2_oxygen_metal2_79,
                  metal2_oxygen_metal1_79, metal1_oxygen_metal2_79, metal1_oxygen_metal1_79]
          return bond




IndentationError: ignored

In [ ]:
# Usage in the same file:
analyzer = BondAnalyzer()
result = analyzer.generate_bond_info("H", "O")

# If you want to use this class in another file, you can import it.
# Assuming you saved this class in a file named "bond_analyzer.py":
# from bond_analyzer import BondAnalyzer